In [1]:
!pip install flask pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=db8ef537d2f60b3cdc97d17c34f3626b51c2d48573371ea239c56b189861bfd2
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [2]:
!pip install pyngrok

In [1]:
!pip install flask flask-ngrok

In [2]:
from flask_ngrok import run_with_ngrok
from flask import Flask,request,render_template

In [7]:
!pip install --upgrade pyngrok

In [11]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=a15ac5b8c23b3699ba5d57a34bddde2530f717bfc17a3b46dd643e1606a3be3b
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [16]:
!pip install flask-cors

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS  # Import CORS
from pyngrok import ngrok
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
from pyspark.ml.tuning import CrossValidatorModel
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Create Flask app
app = Flask(__name__)

# Enable CORS for all origins (or specify the origins you want to allow)
CORS(app, resources={r"/*": {"origins": ["http://localhost:5173"]}})

# Initialize Spark session
spark = SparkSession.builder \
    .appName("SentimentAnalysisAPI") \
    .master("local[*]") \
    .getOrCreate()

# Load the saved pipeline and model
pipelineModel = PipelineModel.load("/content/drive/MyDrive/sentimental_analysis/pipeline_folder")
cvModel = CrossValidatorModel.load("/content/drive/MyDrive/sentimental_analysis/model_folder")

# Define the label mapping function
def map_prediction(prediction):
    if prediction == 2.0:
        return "Negative"
    elif prediction == 1.0:
        return "Neutral"
    elif prediction == 0.0:
        return "Positive"
    else:
        return "Unknown"

# Register the UDF
map_prediction_udf = udf(map_prediction, StringType())

@app.route('/')
def home():
    return "Hello, World!"

@app.route('/predict', methods=['POST'])
def predict_sentiment():
    data = request.json

    # Check if 'text' is present in the request
    if 'text' not in data:
        return jsonify({"error": "No text field provided"}), 400

    # Convert the JSON data to a Spark DataFrame
    new_data = spark.createDataFrame([(1, data['text'])], ["id", "clean_text"])

    # Transform the new data using the pipeline
    transformed_new_data = pipelineModel.transform(new_data)

    # Make predictions
    predictions = cvModel.transform(transformed_new_data)

    # Apply the UDF to add a human-readable label
    predictions_with_labels = predictions.withColumn("predicted_label", map_prediction_udf("prediction"))

    # Get the result
    result = predictions_with_labels.select("predicted_label").collect()[0]["predicted_label"]

    # Return the result as JSON
    return jsonify({"text": data['text'], "sentiment": result})

if __name__ == '__main__':
    # Set ngrok auth token
    ngrok.set_auth_token('2lBvQQTBJSwgRw2dTqZ1F9vqCAG_4TWPvfo4pzRK4AHkF5tpS')

    # Open a tunnel to the local Flask server
    public_url = ngrok.connect(5000)
    print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

    # Run Flask app
    app.run(port=5000)


 * ngrok tunnel "NgrokTunnel: "https://ac26-34-23-2-127.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Aug/2024 11:26:21] "OPTIONS /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2024 11:26:23] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2024 11:32:05] "OPTIONS /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2024 11:32:07] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2024 11:36:13] "OPTIONS /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2024 11:36:15] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2024 11:40:08] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2024 11:40:09] "GET /favicon.ico HTTP/1.1" 404 -
